for braun

In [ ]:
zcat fastq/220709_braun_s3ATAC.barcode.fastq.gz | head -n 100000 > braun.txt


gzip: stdout: Broken pipe


I have index files:

In [8]:
head whitelist/220521_ted_index1.txt
printf "\n"
head whitelist/220709_braun_s3ATAC.index2.txt
printf "\n"
head whitelist/s3_Tn5_8bp.BCONLY.txt

TAAGGCGA
CGTACTAG
AGGCAGAA
TCCTGAGC
GGACTCCT
TAGGCATG
CTCTCTAC
CAGAGAGG
GAAGAGTA
GGTTAGTT

GATCATGATA
CCATATGCTC
TGGATCGTTA
AGTTACGCCG
GGCTGCGACG
ATGGAGCTAC
GCCAAGGCAA
AGAATTAGCA
CAAGTAGGAC
CTAACCTACC

GAACCGCG
AGGTTATA
TCATCCTT
CTGCTTCC
GGTCACGA
AACTGTAG
GTGAATAT
ACAGGCGC
CATAGAGT
TGCGAGAC


Within braun.txt, I try to find `GATCATGATA`, an example of index 2. It's in the middle. I try to find `TAAGGCGA`, an example of index 1. It's at the front. Finally, I try to find `GAACCGCG`, an example of the plate well barcode. It's at the back. So the order of indices within the barcode is Index 1 - index 2 - well barcode, which makes sense. A full cell barcode whitelist is thus obtained as follows:

In [9]:
module load mawk
cartesian(){
awk -F '\t' \
    -v "bc1_list=$1" \
    -v "bc2_list=$2" \
    -v "bc3_list=$3" '
    {
        if ( FILENAME == bc1_list ) {
            BC1[$0];
        }
        else if ( FILENAME == bc2_list ) {
             BC2[$0];
        } else if ( FILENAME == bc3_list ) {
             BC3[$0];
        }
    } END {
        for (bc1 in BC1) {
           for (bc2 in BC2) {
               for (bc3 in BC3) {
                   print bc1 bc2 bc3
               }
           }
        }
     } ' $1 $2 $3 > $4
}


Lmod has detected the following error:  The following module(s) are
unknown: "mawk"

Please check the spelling or version number. Also try "module spider ..."
It is also possible your cache file is out-of-date; it may help to try:
  $ module --ignore_cache load "mawk"

Also make sure that all modulefiles written in TCL start with the string
#%Module

If you don't understand the warning or error, contact the helpdesk at
hpc@ugent.be 




In [3]:
cartesian whitelist/220521_ted_index1.txt whitelist/220709_braun_s3ATAC.index2.txt whitelist/s3_Tn5_8bp.BCONLY.txt braun_whitelist.txt

In [4]:
head braun_whitelist.txt

TCCTGAGCATAGCATCAATGGATCGA
TCCTGAGCATAGCATCAATTGGACTC
TCCTGAGCATAGCATCAACGTCTGCG
TCCTGAGCATAGCATCAATTCCTGTT
TCCTGAGCATAGCATCAACGGAACTG
TCCTGAGCATAGCATCAACAATTAAC
TCCTGAGCATAGCATCAAGCAATGCA
TCCTGAGCATAGCATCAAAGTTCAGG
TCCTGAGCATAGCATCAACTTATCGG
TCCTGAGCATAGCATCAAACCTTGGC


In [9]:
wc -l braun_whitelist.txt

27648 braun_whitelist.txt


which indeed corresponds to the cartesian product.

We can compose a whitelist for sample 1 as well:

!! note i5 here is wrong! needs to be taken from the bccount instead of the metadata.

In [1]:
cat cell.metadata.tsv | cut -d $'\t' -f 1 | sort -u > s1_i5.txt

In [2]:
cat cell.metadata.tsv | cut -d $'\t' -f 2 | sort -u > s1_tn5.txt

In [3]:
cat cell.metadata.tsv | cut -d $'\t' -f 3 | sort -u > s1_i7.txt

In [7]:
cat s1_i5.txt
cat s1_tn5.txt
cat s1_i7.txt

GGACTTATGG
ACCGGAAGAA
TATTGCAGCT
ATCCGAACGG
CTCAGAATAA
CAATTCGTTA
TGCATGGCCA
AGAGGACTGG
AGGCCGGCCA
GGCATAACCG
TCTCATTGCC
TTGGTTCTCA
CCGAACTATG
TCTCCTCCTG
CTGCATTACG
ACCATAATAA
GTACTGGTTG
TGGCTCATAT
CAGCCATGAT
CGGAGCGTCA
GAACTTGCGG
AACCTATAGT
AATTGAGTTA
CTAACCTACC
TAGTCGGATG
CGCCGATATC
CAAGTAGGAC
GTTCTCTCCT
AATGATGCTC
GCCGGAGCGG
CAATGAGAAG
GGCAGCAGTT
AACGTTCC
AACTGTAG
AAGATACT
AAGTCCAA
AATCCGGA
AATGCCTC
ACACTAAG
ACAGGCGC
ACCTTGGC
ACGCACCT
ACTAAGAT
AGCCTCAT
AGCTCGCT
AGGTTATA
AGTACTCC
AGTTCAGG
ATAATGTG
ATATGGAT
ATCCACTG
ATCTGCCA
ATGAGGCC
ATGGCATG
ATGTAAGT
ATTGTGAA
CAAGCTAG
CAATTAAC
CACGAGCG
CAGTAGGC
CATAGAGT
CCAAGTCT
CCATTCGA
CCGCGGTT
CCGTGAAG
CCTTCACC
CGCCGATC
CGCTATGT
CGGAACTG
CGGACAAC
CGGCGTGA
CGTCTGCG
CGTTAGAA
CTACGACA
CTAGCGCT
CTCACCAA
CTCTCGTC
CTGCTTCC
CTTATCGG
CTTGGTAT
GAACCGCG
GACCTGAA
GACGTCTT
GATCTATC
GATTCTGC
GCAATGCA
GCACGGAC
GCAGAATT
GCCACAGG
GCGCAAGC
GCGGCACA
GCTCATTG
GCTTGTCA
GGACTTGG
GGAGCGTC
GGCATTCT
GGCTTAAG
GGTACCTT
GGTCACGA
GTATGTTC
GTCGGAGC
GTGAATAT
GTGTCGGA
GTTCCAAT


In [11]:
cartesian whitelist/OHS_s3atac_1.s1_i7.txt whitelist/OHS_s3atac_1.s1_i5.txt whitelist/OHS_s3atac_1.s1_tn5.txt whitelist/OHS_s3atac_1.whitelist.txt